In [1]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


BATCH_SIZE = 16
transforms = transforms.Compose([
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])
test_dataset = ImageFolder('F:\\dump ware10 dataset\\dumpware10_images\\dumpware10_images\\224\\224\\TEST',transforms)
test_dataloader = DataLoader(test_dataset,shuffle=False,batch_size = BATCH_SIZE,num_workers=4)

In [2]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from model.MIL_CNN import MyModel
from model.MIL_CNN import *
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = MyModel([1,2,4])
model = model.to(device)

def evaluate_testset():
    labels_list = []
    preds_list = []
    
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images) 
            _, preds = torch.max(outputs, 1) 
            preds = preds.cpu().numpy()
            outputs = outputs.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()
            labels_list.extend(labels)
            preds_list.extend(preds)
        
    test_res = {}
    test_res['test_accuracy'] = accuracy_score(labels_list, preds_list)
    test_res['test_precision'] = precision_score(labels_list, preds_list, average='macro')
    test_res['test_recall'] = recall_score(labels_list, preds_list, average='macro')
    test_res['test_f1-score'] = f1_score(labels_list, preds_list, average='macro')
    
    return test_res

cuda


In [3]:
model = torch.load('model.pth')
model.eval()
print(evaluate_testset())

F:\anaconda\envs\paper1\lib\site-packages\torch\nn\modules\module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


{'test_accuracy': 0.9814169570267132, 'test_precision': 0.9772662464863018, 'test_recall': 0.9696569456751546, 'test_f1-score': 0.9728872661361359}


In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 294,003 trainable parameters
